A0287147U Cai Xiaoqi Jessica

### Task 1: Data Loading and Exploration


#### Data Loading (10 marks)
* Load the Bank Marketing dataset into a Spark DataFrame using PySpark.
* Use the bank-full.csv file provided as the dataset.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

# Create a SparkSession -> creates the entry point to all Spark functionality.
spark = SparkSession.builder \
    .appName("bank_insight") \
    .getOrCreate()

# everything is concatenated together in csv file so need to define schema
bank_schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", IntegerType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", IntegerType(), True),
    StructField("campaign", IntegerType(), True),
    StructField("pdays", IntegerType(), True),
    StructField("previous", IntegerType(), True),
    StructField("poutcome", StringType(), True),
    StructField("y", StringType(), True)  # Target variable (yes/no)
])

# Define the path to dataset.
data_path = "/FileStore/tables/bank_full.csv"

# use sparkession to read data -> working with dataFrames not RDDs
df = spark.read.csv(data_path, header=True, schema=bank_schema, sep = ";")

# Write the DataFrame to a Parquet file in DBFS (overwrite mode)
df.write.mode("overwrite").parquet("/tmp/bank_parquet")

# Write the DataFrame to a CSV file in DBFS (overwrite mode and include header)
df.write.mode("overwrite").option("header", "true").csv("/tmp/bank_csv")


#### Exploration Using DataFrame API and Spark SQL (15 marks)
* Display the schema and show a sample (5 rows) of the data using the
DataFrame API.

In [0]:
# display schema
df.printSchema()
df.show(5, truncate=False)

# Cache the final DataFrame to speed up any subsequent operations.
df = df.cache()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|job         |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|y  |
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+

* Write and execute at least two Spark SQL queries. For instance, use one query
for demographic features (age, balance) and another for campaign attributes
(campaign duration), each calculating summary statistics (count, average, min,
max).

In [0]:
# Create a temporary view so you can run SQL queries on the DataFrame.
df.createOrReplaceTempView("bank_data")

# query for age and balance
spark.sql("""
    SELECT
        COUNT(*) AS total_records,
        AVG(age) AS avg_age,
        MIN(age) AS min_age,
        MAX(age) AS max_age,
        AVG(balance) AS avg_balance,
        MIN(balance) AS min_balance,
        MAX(balance) AS max_balance
    FROM bank_data
""").show()

# query for call duration and campaign contacts
spark.sql("""
    SELECT
        COUNT(*) AS total_calls,
        AVG(duration) AS avg_duration,
        MIN(duration) AS min_duration,
        MAX(duration) AS max_duration,
        AVG(campaign) AS avg_campaign,
        MIN(campaign) AS min_campaign,
        MAX(campaign) AS max_campaign
    FROM bank_data
""").show()


+-------------+-----------------+-------+-------+------------------+-----------+-----------+
|total_records|          avg_age|min_age|max_age|       avg_balance|min_balance|max_balance|
+-------------+-----------------+-------+-------+------------------+-----------+-----------+
|        45211|40.93621021432837|     18|     95|1362.2720576850766|      -8019|     102127|
+-------------+-----------------+-------+-------+------------------+-----------+-----------+

+-----------+-----------------+------------+------------+-----------------+------------+------------+
|total_calls|     avg_duration|min_duration|max_duration|     avg_campaign|min_campaign|max_campaign|
+-----------+-----------------+------------+------------+-----------------+------------+------------+
|      45211|258.1630797814691|           0|        4918|2.763840658246887|           1|          63|
+-----------+-----------------+------------+------------+-----------------+------------+------------+



* In your markdown cells, include a brief discussion on how these exploratory steps help in understanding the dataset.

Inspecting the schema ensures all columns have the correct data types, which prevents issues during transformations. 
Viewing the first few rows confirms that the data is properly loaded and parsed.


The first SQL query provides insights into customer demographics.
The average customer age is approximately **40.94 years**, with a range from **18 to 95**, indicating a broad age distribution.  
The average account balance is around **1362.27**, ranging from **-8019 to 102127** (currency unit not specified, but likely in euros). The wide spread and presence of negative balances suggest financial diversity, from wealthy individuals to those potentially in debt. These insights help identify customer segments that may behave differently.

The second SQL query focuses on marketing campaign effectiveness.
The average call duration is **258 seconds** (~4.3 minutes), ranging from **0 to 4918** seconds. A `duration = 0` likely means no engagement, which is important for modeling.  
The average number of contacts per client during the campaign is **2.76**, with a maximum of **63**. Extremely high contact counts may indicate over-marketing, potentially leading to customer fatigue.

These exploratory insights give us a better understanding of our dataset’s structure and possible feature relevance. Features like **balance**, **age**, **duration**, and **campaign frequency** may be strong predictors of whether a customer subscribes to a term deposit, and could guide our future feature engineering.

These exploratory steps highlight which features (e.g., **age**, **balance**, **duration**, **campaign**) are likely influential in predicting term deposit subscriptions. They also guide feature engineering, help assess data quality, and inform business strategies such as customer segmentation and engagement timing.

### Task 2: Feature Engineering with Window Functions

#### Target Variable Transformation (5 marks)
* Convert the target variable (subscription) into a binary label (1 for “yes”, 0 for “no”).


In [0]:
# use StringIndexer to turn categorical string value into numeric indices
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="y", outputCol="label")
df = indexer.fit(df).transform(df)

# Show results to verify
df.select("y", "label").show(10)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-2902284467641502>:5
      2 from pyspark.ml.feature import StringIndexer
      4 indexer = StringIndexer(inputCol="y", outputCol="label")
----> 5 df = indexer.fit(df).transform(df)
      7 # Show results to verify
      8 df.select("y", "label").show(10)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/spark/python/pyspark/ml/base.py:205, in Estimator.fit(self, dataset, params)
    203         return self.copy(params)._fit(dataset)
    204     else:
--> 205         return self._fit(dataset)
    206 else:
    207     raise TypeError(
    2

Window Functions (10 marks)
* Use Spark SQL window functions to create at least one meaningful new feature from the existing data.  
* In a markdown cell, clearly explain the logic and intended benefit of this new feature.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

# Define window: group rows by job
job_window = Window.partitionBy("job")

# Create the new column with average balance per job
df = df.withColumn("avg_balance_by_job", avg("balance").over(job_window))

# Show a few records
df.select("job", "balance", "avg_balance_by_job").show(10)


+------+-------+------------------+
|   job|balance|avg_balance_by_job|
+------+-------+------------------+
|admin.|    270| 1135.838909301876|
|admin.|    390| 1135.838909301876|
|admin.|     13| 1135.838909301876|
|admin.|   -372| 1135.838909301876|
|admin.|     39| 1135.838909301876|
|admin.|    506| 1135.838909301876|
|admin.|      0| 1135.838909301876|
|admin.|   -171| 1135.838909301876|
|admin.|    -76| 1135.838909301876|
|admin.|      0| 1135.838909301876|
+------+-------+------------------+
only showing top 10 rows



Feature: `avg_balance_by_job`

This feature helps assess how a customer is doing financially compared to others in the same occupation. For example, if the average balance for admins is 1,135 but one admin only has 270, it suggests they may be financially underperforming relative to their peers, which could influence their financial decisions, such as whether to commit to a term deposit. While this feature is primarily useful for within-job comparison, it can also support indirect comparisons across jobs when used together with the encoded job feature. In doing so, the model can learn broader patterns. For instance, that some occupations generally have higher financial baselines than others, which may further refine its predictions.


#### Additional Features (7 marks)
* Engineer at least two additional features based on customer demographics and campaign data.
* Consider features that might capture customer behaviour or campaign
characteristics.

In [0]:
from pyspark.sql.functions import when, col

# Create the new feature long_call_no_loan 
df = df.withColumn(
    "long_call_no_loan",
    when(
        (col("duration") > 260) & # since we know that the average call duration is 258.16, then 260 should be considered long
        (col("housing") == "no") &
        (col("loan") == "no"),
        1
    ).otherwise(0)
)

# Show a few records
df.groupBy("long_call_no_loan").count().show()

+-----------------+-----+
|long_call_no_loan|count|
+-----------------+-----+
|                1| 5730|
|                0|39481|
+-----------------+-----+



In [0]:
from pyspark.sql.functions import col

df = df.withColumn(
    "balance_to_age_ratio",
    (col("balance") / (col("age") + 1))  # +1 to avoid division by 0
)

df.select("age", "balance", "balance_to_age_ratio").show(10, truncate=False)

+---+-------+--------------------+
|age|balance|balance_to_age_ratio|
+---+-------+--------------------+
|58 |2143   |36.32203389830509   |
|44 |29     |0.6444444444444445  |
|33 |2      |0.058823529411764705|
|47 |1506   |31.375              |
|33 |1      |0.029411764705882353|
|35 |231    |6.416666666666667   |
|28 |447    |15.413793103448276  |
|42 |2      |0.046511627906976744|
|58 |121    |2.0508474576271185  |
|43 |593    |13.477272727272727  |
+---+-------+--------------------+
only showing top 10 rows



#### Documentation (3 marks)
* In a markdown cell, justify your new features and explain how they could
improve prediction performance.

Additional Feature 1: Long Call and No Loan (Binary)

Based on our exploratory analysis in Task 1, we found that the average call duration was approximately 258 seconds. Therefore, we define "long" calls as those lasting more than 260 seconds, highlighting customers who were more engaged during the call.

This feature captures customers who had a long call (**duration > 260 seconds**) and have **no housing or personal loans**. The assumption is that these individuals are both **financially unburdened** and **genuinely interested**, making them more likely to subscribe to a term deposit. This interaction feature combines **behavioral engagement** and **financial freedom** into a single binary indicator, helping the model identify high-potential customers more effectively.

This feature captures customers who had a **long call** (**duration > 260 seconds**) and have **no housing or personal loans**. The assumption is that these individuals are both financially unburdened and genuinely interested, making them more likely to subscribe to a term deposit. It combines behavioral engagement with financial freedom into a single binary indicator.

There are 5730 entries that fits this long call no loan criteria and 39481 that does not. This means approximately **12.7%** of the customers meet the criteria — a meaningful but not overwhelming subset of the data, ensuring the feature provides useful variation for model learning.


Additional Feature 2: balance to age ratio

The `balance_to_age_ratio` feature captures a customer's financial standing relative to their age. While raw account balance provides a general sense of wealth, it lacks context. For example, a €2000 balance may represent financial strength for a 25-year-old but might be considered modest for someone nearing retirement. By normalizing balance over age, this feature helps the model better understand whether a customer is financially well-off for their life stage. Additionally, negative values reflect customers who are in debt, which can be a strong signal of low likelihood to subscribe to a term deposit. This feature adds interpretability and nuance to financial profiling in the model.


### Task 3: Pipeline Construction and Model Training


#### Pipeline Construction (12 marks)
* Build a PySpark ML pipeline that includes:  
1) Data transformations (including any applied via Spark SQL, if relevant).

  2) A VectorAssembler to combine features.

  3) A classifier (e.g. Logistic Regression) for predicting term deposit
subscription.

In [0]:
from pyspark.sql.functions import col, to_date, month, round as spark_round
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

# 1. data transformation

# all categorical features to be indexed and one-hot encoded
categorical_features = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]

# StringIndexers 
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep")
    for col in categorical_features
]

# OneHotEncoders
encoders = [
    OneHotEncoder(inputCol=col + "_index", outputCol=col + "_vec")
    for col in categorical_features
]

# 2. vectorAssembler to combine all features into a single feature vector:

# all numerical features 
numeric_features = [
    "age", "balance", "day", "duration", "campaign", "pdays", "previous",
    "avg_balance_by_job", "balance_to_age_ratio", "long_call_no_loan" # even though long_call_no_loan is categorical, it is already in binary 
]

# Assemble all features into a single vector
assembler = VectorAssembler(
    inputCols=[col + "_vec" for col in categorical_features] + numeric_features,
    outputCol="features"
)

# 3. Initialise the classifier (Logistic Regression)
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Construct the full pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])


#### Hyperparameter Tuning (7 marks)
* Implement cross-validation with a parameter grid (tune at least two
hyperparameters).
* Adjust the grid size and number of folds if necessary to work within the
limitations of Community Edition.

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Split the data (80% train, 20% test)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Set up a parameter grid for hyperparameter tuning
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0]) \
    .build()
# regParam -> Strength of regularization (prevent overfitting), trying both weak (0.01) and strong (0.1) regularization
# elasticNetParam -> L1 vs L2 regularization (Lasso vs Ridge), just try Ridge Regression (L2 regularization)

# Define evaluator using area under the ROC curve
evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
# help pick the best model

# Set up CrossValidator with 3-fold cross-validation
cv = CrossValidator(
    estimator=pipeline,  # The entire ML pipeline
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3  # 3-fold cross-validation, reasonable for databricks CE
)

# Train the model using cross-validation
cvModel = cv.fit(train_df)

#### Documentation (6 marks)
* In a markdown cell, describe the stages of your pipeline.

The machine learning pipeline consists of the following stages:

1. StringIndexers: Categorical features such as `job`, `marital`, `education`, and others are transformed into numerical indices using `StringIndexer`. This is necessary for compatibility with Spark ML models.

2. OneHotEncoders: The indexed categorical features are further encoded into binary vectors using `OneHotEncoder`, allowing the model to treat them as distinct categories rather than ordinal numbers.

3. **VectorAssembler**: All numerical features (including both original and engineered features like `avg_balance_by_job`, `balance_to_age_ratio`, and `long_call_no_loan`) and encoded categorical features are assembled into a single feature vector named `features`.

4. **Logistic Regression**: A Logistic Regression model is used to classify whether a customer will subscribe to a term deposit (`label` = 1 for "yes", 0 for "no"). It supports regularization to prevent overfitting and works well for binary classification problems.

5. **CrossValidator**: A 3-fold cross-validation is used to evaluate model performance across multiple parameter settings, allowing the best combination of hyperparameters to be selected based on Area Under ROC.


* In a markdown cell, explain how distributed processing in Spark benefits your ML
pipeline and how hyperparameter tuning improves model performance,
including any adjustments made due to resource constraints.

Benefits of Distributed Processing in Spark: PySpark's MLlib runs transformations and model training across distributed workers. This allow for efficient handling of large datasets like `bank-full.csv`, faster feature transformation, model training, and evaluation using Spark clusters, and support for pipeline parallelism and lazy evaluation to minimize resource waste.

Hyperparameter Tuning Justification: I used cross-validation to tune two parameters: `regParam` (regularization strength) and `elasticNetParam` (balance between L1 and L2 regularization). This improves generalizability and reduces overfitting. Due to Databricks Community Edition's limited resources, I used a smaller grid and 3-fold CV instead of 5 or 10 folds. This balances computation time with model quality, making it efficient yet effective for our environment.


### Task 4: Model Evaluation and Innovation and Advanced Insights

#### Model Evaluation (15 marks)
* Split your data into training and test sets (80:20).
* Evaluate your model on the test set using at least three metrics (e.g. accuracy,
precision, recall, F1 score, or area under ROC).
* Clearly display the results.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Split dataset into training (80%) and test (20%)
# already done it in task 3 before training the model and cross validation

# Make predictions on test data
predictions = cvModel.transform(test_df)

# Evaluate model performance

# AUC (Area Under ROC)
auc_evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = auc_evaluator.evaluate(predictions)
print(f"Test AUC: {auc:.4f}")

# Accuracy
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.4f}")

# F1 Score
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(predictions)
print(f"Test F1 Score: {f1:.4f}")

# Show prediction samples
predictions.select("label", "prediction", "probability").show(10, truncate=False)

Test AUC: 0.9079
Test Accuracy: 0.9026
Test F1 Score: 0.8868
+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0.0  |0.0       |[0.9290427370186412,0.07095726298135885]|
|0.0  |0.0       |[0.9161813366000454,0.08381866339995458]|
|1.0  |1.0       |[0.39529415882975855,0.6047058411702415]|
|0.0  |0.0       |[0.8734226925101197,0.12657730748988028]|
|1.0  |1.0       |[0.39236672113685717,0.6076332788631429]|
|0.0  |0.0       |[0.6309218125032943,0.36907818749670573]|
|1.0  |0.0       |[0.7180794911308055,0.2819205088691945] |
|0.0  |0.0       |[0.8512903206231359,0.14870967937686408]|
|0.0  |0.0       |[0.7386379230098638,0.26136207699013625]|
|0.0  |0.0       |[0.9110928369439552,0.08890716305604485]|
+-----+----------+----------------------------------------+
only showing top 10 rows



Test AUC: 0.9079

Test Accuracy: 0.9026

Test F1 Score: 0.8868


#### Innovation and Advanced Insights (10 marks)
* In a markdown cell, propose an innovative or creative extension to your solution. This might include:  
  1) Novel feature engineering techniques,
  
  2) Experimenting with an alternative machine learning algorithm,
  
  3) Creative visualisations of model performance or data distributions.


Innovation: Experimenting with a Tree-Based Algorithm (Random Forest)

While Logistic Regression is interpretable and efficient, it assumes linear relationships between features and the outcome. As an extension, I propose experimenting with a Random Forest Classifier, which is a tree-based ensemble method that can model non-linear interactions and complex feature relationships more effectively. Random Forests are also robust to outliers and can naturally handle categorical variables through feature splitting. Importantly, they provide feature importance scores, which offer insights into which variables are most influential in predicting term deposit subscriptions — a valuable asset for guiding business decisions. Given the relatively high class imbalance in the dataset, Random Forests can be further tuned using class weighting or sampling techniques to boost F1 score and recall. This extension could improve predictive performance and offer more nuanced interpretability beyond what Logistic Regression provides.



* In a markdown cell, analyse your model's results. Discuss aspects like feature
importance or model interpretability and explain how they add value to your
assignment.

The final model achieved strong performance on the test set:

- **Accuracy**: 90.3%
- **F1 Score**: 88.7%
- **AUC**: 90.8%

These results suggest the model is well-calibrated and generalizes well to unseen data. The high AUC reflects strong class separation, while the high F1 score indicates balanced precision and recall, which is crucial in a real-world marketing context where false positives and false negatives can be costly.

Although Logistic Regression is less expressive than tree-based models, its transparency allows us to understand how each feature contributes to the outcome. Features like `duration`, `balance_to_age_ratio`, and `long_call_no_loan` likely played important roles by combining behavioral and financial indicators.

In conclusion, this pipeline offers a practical and interpretable solution. With additional time or computational resources, further improvements could be achieved through model ensembling, richer visualizations, and more advanced feature extraction.
